In [1]:
import findspark
findspark.init()

In [2]:
from __future__ import print_function

import sys
from random import random
from operator import add

from pyspark.sql import SparkSession

In [3]:
# read text file
def read_text_file():
    book = ""
    with open('data/Shakespeare.txt', 'r') as f:
        #read and remove empty space from the last of the text
        book = f.read().strip()
    return book    

In [4]:
# remove header
def remove_hearder():
    all_text = read_text_file()
    
    # text to array -> took each line of the text book to each element of array
    text_to_array = all_text.split("\n")
    
    # take all the lines after 245
    text_to_array = text_to_array[244: ]
    
    # add all lines toghater
    all_text = ("\n".join(text_to_array)).strip()
    
    return all_text

In [5]:
import re

# remove the autogenerated text
def remove_automatic_text():
    all_text = remove_hearder()
    
    # remove the auto generated text from the text by regular expression
    all_text = re.sub(r'<<[^>]*>>', '', all_text)
    
    return all_text

In [6]:
# split into segments

def split_to_segments():
    all_text = remove_automatic_text()
    
    # split all plays by THE END
    all_plays = all_text.strip().split("THE END")
    
    # remove the last portion End of .... 
    all_plays.pop()
    
    all_plays = [plays.strip() for plays in all_plays]
    
    return all_plays

In [7]:
# the play count function that execute for each of the parallel call 
def play_counts(play):
    
    # summary dictionary 
    summary = {}
    
    # split a single play into each line
    plays_arr = play.strip().split("\n")

    #remove the line which countains 0 or 1 characters
    plays_arr = [line for line in plays_arr if len(line.strip()) > 1]
    
    # remove the number before play and the title from original play from the line list
    play_title = plays_arr[1]
    plays_arr = plays_arr[2:]
    
    # worlds list
    words_arr = (" ".join(plays_arr)).split(" ")
    
    # remove all emply space from the sentence
    words_arr = [word for word in words_arr if len(word.strip())>0]
    
    return {"title": play_title, "lines": len(plays_arr), "words": len(words_arr)}

In [29]:
import pandas as pd

# initialize spark session
spark = SparkSession.builder.appName("Shakespeare Book Summary").getOrCreate()

# get all segments/plays
all_plays = split_to_segments()

# execute parallel call
summary = spark.sparkContext.parallelize(all_plays, len(all_plays)).map(play_counts)

# convert to data frame for easy access and sorting
df = summary.toDF().toPandas()

# sort by line number
df = df.sort_values(by="lines", ascending=False)

for  ind, row in df.iterrows():
    print(f'{row["title"]}, {row["lines"]} lines, {row["words"]} words')
    
spark.stop()

C:\ProgramData\Anaconda3\lib\site-packages\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


THE TRAGEDY OF HAMLET, PRINCE OF DENMARK, 4161 lines, 31946 words
KING RICHARD III, 4152 lines, 31084 words
THE TRAGEDY OF CORIOLANUS, 3909 lines, 29208 words
THE TRAGEDY OF ANTONY AND CLEOPATRA, 3841 lines, 26453 words
CYMBELINE, 3837 lines, 28782 words
THE HISTORY OF TROILUS AND CRESSIDA, 3632 lines, 27528 words
THE TRAGEDY OF KING LEAR, 3629 lines, 27505 words
THE TRAGEDY OF OTHELLO, MOOR OF VENICE, 3628 lines, 27856 words
KING HENRY THE EIGHTH, 3487 lines, 25810 words
THE WINTER'S TALE, 3366 lines, 25943 words
THE SECOND PART OF KING HENRY THE SIXTH, 3314 lines, 26773 words
THE LIFE OF KING HENRY THE FIFTH, 3311 lines, 27437 words
SECOND PART OF KING HENRY IV, 3282 lines, 27622 words
THE TRAGEDY OF ROMEO AND JULIET, 3280 lines, 25780 words
THE THIRD PART OF KING HENRY THE SIXTH, 3186 lines, 25822 words
THE FIRST PART OF KING HENRY THE FOURTH, 3072 lines, 25710 words
THE FIRST PART OF HENRY THE SIXTH, 3052 lines, 22766 words
KING RICHARD THE SECOND, 2977 lines, 23290 words
ALLS WELL